In [ ]:
dataDirectory = "dat/ReArm.lnk/ReArm_C1P02"

# load goodFiles.txt into a list
goodFiles = []
with open("../" + dataDirectory + "/goodFiles.txt", "r") as f:
    for line in f:
        goodFiles.append(line.strip())

print("goodFiles.txt contains {} files".format(len(goodFiles)))

In [ ]:
expectedStreams = [
    ["EuroMov-Markers-Kinect", "Markers"],
    ["NIC-EEG", "EEG"],
    ["NIC-Markers", "Markers"],
    ["NIC-Quality", "Quality"],
    ["EuroMov-Mocap-Kinect", "MoCap"],
    ["Mouse", "Markers"],
    ["MouseToNIC", "Markers"],
    ["Mouse", "MoCap"],
    ["Oxysoft", "NIRS"],
    ["NIC-Accelerometer", "Accelerometer"],
    ["Oxysoft Event", "Event"],
    #["Oxysoft Event", "toto"],
]

# Explore the synchrony of the XDF and CSV files


In [ ]:
%matplotlib qt
# to get zoomable figures

import numpy as np
from datetime import datetime
import os
import pyxdf


xdf_fullFname = goodFiles[17] # xdf file
#fullFname = goodFiles[0] # csv file

def file_parts(fullFname):
    """ Split a full filename into path, filename and extension"""
    fpath = os.path.dirname(fullFname)
    fname = os.path.basename(fullFname)
    fname, extension = os.path.splitext(fname)
    return fpath, fname, extension

# analyse the extension of the file
fpath , fname, extension = file_parts(xdf_fullFname)

# split the filename into tokens
tk = fname.split("_")


# List all the streams in the XDF file

In [ ]:
# load the full xdf file to show the streams
data, header = pyxdf.load_xdf(
    xdf_fullFname, synchronize_clocks=True, dejitter_timestamps=False
)
# list all the streams in the file
print("Found {} streams:".format(len(data)))
for ix, stream in enumerate(data):
    print(
        "  {:02d}: {} {} - {} - shape {} at {} Hz (effective {:5.2f} Hz)".format(
            stream["info"]["stream_id"],
            ix + 1,
            stream["info"]["name"][0],
            stream["info"]["type"][0],
            (int(stream["info"]["channel_count"][0]), len(stream["time_stamps"])),
            stream["info"]["nominal_srate"][0],
            stream["info"]["effective_srate"],
        )
    )

# check if all the expected streams are present
foundStreams = []
missingStreams = []
for i in range(len(expectedStreams)):
    expectedStream = expectedStreams[i]
    found = False
    for ix, stream in enumerate(data):
        if stream["info"]["name"][0] == expectedStream[0] and stream["info"]["type"][0] == expectedStream[1]:
            found = True
            foundStreams.append(expectedStream)
    if not found:
        missingStreams.append(expectedStream)

# print the missing streams
print("Missing streams:")
if len(missingStreams) == 0:
    print("  None")
else:
    for stream in missingStreams:
        print("  {}".format(stream))

# print the found streams
print("Found streams:")
for stream in foundStreams:
    print("  {}".format(stream))

# Explore the synchrony of the XDF and CSV files for the Kinect data 

The CSV and XDF files contain the exact same information, but the XDF file has timestamps relative to the start of the kinect recording, while the CSV file has absolute timestamps (i.e., from the host computer).
If the CSV to XDF delay is constant, this can be used as a correction factor for the (wrong) timestamps of the kinect stream.

In [ ]:
# load the EuroMov-Mocap-Kinect stream
data, header = pyxdf.load_xdf(
    filename=xdf_fullFname,
    select_streams=[{"type": "MoCap", "name": "EuroMov-Mocap-Kinect"}],
    synchronize_clocks=True,
    dejitter_timestamps=False,  # to get the raw timestamps to compare with the CSV
)
data_XDF = data[0]

# get the corresponding kinect file name
kinectFname = (
    tk[0]
    + "_"
    + tk[1]
    + "_"
    + tk[2]
    + "_"
    + tk[3]
    + "_"
    + tk[4]
    + "_k.csv"
)
fullFname_csv = os.path.join(fpath, kinectFname)

# get the corresponding data from the CSV file
data_csv = np.loadtxt(fullFname_csv, skiprows=3, delimiter=",", dtype=float)

# get the timestamps from the two files
timestampCSV_sec = data_csv[:, 0] / 1000
timestampXDF_sec = data_XDF["time_stamps"]

# get the (median) delay between the two files
delay = np.median(timestampXDF_sec - timestampCSV_sec)

# get the distribution of the delays
delays = timestampXDF_sec - timestampCSV_sec - delay


In [ ]:
# do the plots
import matplotlib.pyplot as plt

# make a boxplot of the distribution of the delays
fig = plt.figure()
plt.boxplot(delays, vert=False)
plt.xlabel("Delay (s)")
# add a title with the median delay
plt.title("Distribution of the delays")
plt.show()

# make a plot of the timestampXDF and timestampCSV
fig = plt.figure()
plt.plot(timestampXDF_sec - timestampXDF_sec[0], "o", label="XDF")
plt.plot(timestampCSV_sec - timestampCSV_sec[0], "o", label="CSV")
plt.ylabel("TimeStamp (s)")
plt.xlabel("Line number")
plt.legend()
plt.show()

# make a plot of the distribution of the delays
fig = plt.figure()
plt.plot(delays * 1000, "o")
plt.ylabel("Delay (milliseconds)")
plt.xlabel("Line number")
plt.show()

# Compute the delay between the XDF and CSV files for the Kinect data



In [ ]:
def getKinectDelay(xdf_fullFname):
    """
    Get the delay between the XDF file and the corresponding CSV file for the kinect data
    """

    fpath, fname, extension = file_parts(xdf_fullFname)

    # split the filename into tokens
    tokens = fname.split("_")

    if extension != ".xdf":
        #print("Error: the file extension is not .xdf")
        return None

    # load the EuroMov-Mocap-Kinect stream
    data, header = pyxdf.load_xdf(
        filename=xdf_fullFname,
        select_streams=[{"type": "MoCap", "name": "EuroMov-Mocap-Kinect"}],
        synchronize_clocks=True,
        dejitter_timestamps=False,  # to get the raw timestamps to compare with the CSV
    )
    data_XDF = data[0]

    # get the corresponding kinect file name
    kinectFname = (
        tokens[0]
        + "_"
        + tokens[1]
        + "_"
        + tokens[2]
        + "_"
        + tokens[3]
        + "_"
        + tokens[4]
        + "_k.csv"
    )
    fullFname_csv = os.path.join(fpath, kinectFname)

    # get the corresponding data from the CSV file
    data_csv = np.loadtxt(fullFname_csv, skiprows=3, delimiter=",", dtype=float)

    # get the timestamps from the two files
    timestampCSV_sec = data_csv[:, 0] / 1000
    timestampXDF_sec = data_XDF["time_stamps"]

    # get the (median) delay between the two files
    delay = np.median(timestampXDF_sec - timestampCSV_sec)

    # get the distribution of the delays
    delays_distribution = timestampXDF_sec - timestampCSV_sec - delay

    return {
        "delay": delay,
        "delays_distribution": delays_distribution,
        "fullFname": xdf_fullFname,
    }



# get the delay for all the files
delays = []
for fullFname in goodFiles:
    delay = getKinectDelay(fullFname)
    if delay is not None:
        delays.append([delay, fullFname])

# print the delays
print("Delays:")
for delay in delays:
    print(delay[0]["delay"], delay[0]["fullFname"])

# make a boxplot of the distribution of the delays
fig = plt.figure()
plt.boxplot([delay[0]["delays_distribution"] for delay in delays], vert=False)
plt.xlabel("Delay (s)")
plt.title("Distribution of the delays")
plt.show()

# make a plot of the distribution of the delays
fig = plt.figure()
for delay in delays:
    plt.plot(delay[0]["delays_distribution"] * 1000, "o")
plt.ylabel("Delay (milliseconds)")
plt.xlabel("Line number")
plt.show()

# Compute the delay between the XDF and CSV files for the mouse data

In [ ]:
def getMouseDelay(xdf_fullFname):
    """
    Get the delay between the XDF file and the corresponding CSV file for the mouse data
    """

    debug_getMouseDelay = False

    fpath, fname, extension = file_parts(xdf_fullFname)

    tk = fname.split("_")

    if extension != ".xdf":
        # print("Error: the file extension is not .xdf")
        return None
    # load the EuroMov-Mocap-Kinect stream
    data, header = pyxdf.load_xdf(
        filename=xdf_fullFname,
        select_streams=[{"type": "Markers", "name": "Mouse"}],
        synchronize_clocks=True,
        dejitter_timestamps=False,  # to get the raw timestamps to compare with the CSV
    )
    data_XDF = data[0]

    # get the corresponding mouse file name
    if tk[4] == "r":
        mouseFnameEnd = "_l_m_sau_p.csv"
    elif tk[4] == "c":
        mouseFnameEnd = "_l_m_p.csv"

    mouseFname = (
        tk[0] + "_" + tk[1] + "_" + tk[2] + "_" + tk[3] + "_" + tk[4] + mouseFnameEnd
    )

    fullFname_csv = os.path.join(fpath, mouseFname)

    # get the corresponding data from the CSV file
    mouse_csv = np.loadtxt(
        fullFname_csv, skiprows=3, max_rows=10, quotechar='"', delimiter=",", dtype=str
    )

    csv_timestamps = mouse_csv[:, 1].astype(float) / 1000

    if debug_getMouseDelay:
        print(csv_timestamps.shape)
        # print all csv timestamps and markers
        for i in range(len(csv_timestamps)):
            print(csv_timestamps[i], mouse_csv[i, 2])

        print(data_XDF["time_stamps"].shape)
        # print all xdf timestamps and markers
        for i in range(len(data_XDF["time_stamps"])):
            print(data_XDF["time_stamps"][i], data_XDF["time_series"][i])

    # restrict the xdf data to the first 10 markers
    xdf_timestamps = data_XDF["time_stamps"][:10]
    xdf_markers = data_XDF["time_series"][:10]

    # get the timestamps from the two files
    timestampCSV_sec = csv_timestamps
    timestampXDF_sec = xdf_timestamps

    # get the (median) delay between the two files
    delay = np.median(timestampXDF_sec - timestampCSV_sec)

    # get the distribution of the delays
    delays_distribution = timestampXDF_sec - timestampCSV_sec - delay

    if debug_getMouseDelay:
        # get the diff of the timestamps of the csv file adn the xdf file
        csv_diff = np.diff(timestampCSV_sec)
        xdf_diff = np.diff(timestampXDF_sec)
        print(csv_diff - xdf_diff)

        # do the plots
        import matplotlib.pyplot as plt

        # make a boxplot of the distribution of the delays
        fig = plt.figure()
        plt.boxplot(delays_distribution, vert=False)
        plt.xlabel("Delay (s)")
        # add a title with the median delay
        plt.title("Distribution of the delays")
        plt.show()

        # make a plot of the timestampXDF and timestampCSV
        fig = plt.figure()
        plt.plot(timestampXDF_sec - timestampXDF_sec[0], "o", label="XDF")
        plt.plot(timestampCSV_sec - timestampCSV_sec[0], "o", label="CSV")
        plt.ylabel("TimeStamp (s)")
        plt.xlabel("Line number")
        plt.legend()

    return {
        "delay": delay,
        "delays_distribution": delays_distribution,
        "fullFname": xdf_fullFname,
    }


getMouseDelay(goodFiles[17])
getMouseDelay(goodFiles[8])

In [ ]:
# get the delay for all the files
mouse_delays = []
for fullFname in goodFiles:
    delay = getMouseDelay(fullFname)
    if delay is not None:
        mouse_delays.append([delay, fullFname])

# print the delays
print("Mouse Delays:")
for delay in mouse_delays:
    print(delay[0]["delay"], delay[0]["fullFname"])

# make a boxplot of the distribution of the delays
fig = plt.figure()
plt.boxplot([delay[0]["delays_distribution"] for delay in mouse_delays], vert=False)
plt.xlabel("Delay (s)")
plt.title("Distribution of the delays")
plt.show()

# make a plot of the distribution of the delays
fig = plt.figure()
for delay in mouse_delays:
    plt.plot(delay[0]["delays_distribution"] * 1000, "o")
plt.ylabel("Delay (milliseconds)")
plt.xlabel("Line number")
plt.show()

# Get the kinect-to-mouse delay

In [ ]:
fileName = goodFiles[8]
fileName  = goodFiles[17]

if False: 
    kinect_markers = pyxdf.load_xdf(
        filename=fileName,
        select_streams=[{"type": "Markers", "name": "EuroMov-Markers-Kinect"}],
        synchronize_clocks=True,
        dejitter_timestamps=False,  # to get the raw timestamps to compare with the CSV
    )
    kinect_markers = kinect_markers[0][0]


    # print("Kinect markers")
    # for i in range(len(kinect_markers["time_stamps"])):
    #    # print the timestamp and the marker
    #     print(kinect_markers["time_stamps"][i], kinect_markers["time_series"][i][0])


    mouse_markers = pyxdf.load_xdf(
        filename=fileName,
        select_streams=[{"type": "Markers", "name": "Mouse"}],
        synchronize_clocks=True,
        dejitter_timestamps=False,  # to get the raw timestamps to compare with the CSV
    )
    mouse_markers = mouse_markers[0][0]

else:
    # load the whole xdf file
    data, header = pyxdf.load_xdf(
        filename=fileName,
        synchronize_clocks=True,
        dejitter_timestamps=False,  # to get the raw timestamps to compare with the CSV
    )

    # find the mouse and kinect markers
    kinect_markers = [
        stream for stream in data if stream["info"]["name"][0] == "EuroMov-Markers-Kinect" and stream["info"]["type"][0] == "Markers"
    ]
    kinect_markers = kinect_markers[0]

    mouse_markers = [stream for stream in data if stream["info"]["name"][0] == "Mouse" and stream["info"]["type"][0] == "Markers"]
    mouse_markers = mouse_markers[0]

# get the markers
nbKinectMarkers = len(kinect_markers["time_stamps"])
nbMouseMarkers = len(mouse_markers["time_stamps"])

# check whether the mouse and kinect markers are from the same hostname
kinect_hostname = kinect_markers["info"]["hostname"][0]
mouse_hostname = mouse_markers["info"]["hostname"][0]
if kinect_hostname == mouse_hostname:
    print(
        "The mouse and kinect markers are from the same computer clock: {}".format(
            kinect_hostname
        )
    )
else:
    print("CAUTION: the mouse and kinect markers are not from the same computer clock")
    print("  Kinect hostname: {}".format(kinect_hostname))
    print("  Mouse hostname: {}".format(mouse_hostname))


# print("Mouse markers")
# for i in range(nbMouseMarkers):
#     # print the timestamp and the marker
#     print(mouse_markers["time_stamps"][i], mouse_markers["time_series"][i][0])

# get the delay
kinect_delay = getKinectDelay(fileName)["delay"]
mouse_delay = getMouseDelay(fileName)["delay"]

# get the delay from the kinect to the mouse
kinect_to_mouse_delay = mouse_delay - kinect_delay

# get the delays as daytimes
kinect_datetime = datetime.fromtimestamp(kinect_delay)
mouse_datetime = datetime.fromtimestamp(mouse_delay)
kinect_to_mouse_daytime = datetime.fromtimestamp(kinect_to_mouse_delay)

# get the kinect_to_mouse delay as a difference between two daytimes
kinect_to_mouse_daytime = kinect_datetime - mouse_datetime


# print the delays as number and daytimes
print("Delays:")
print("  Kinect delay: {} ({})".format(kinect_delay, kinect_datetime))
print("  Mouse delay: {} ({})".format(mouse_delay, mouse_datetime))
print(
    "  Kinect to mouse delay: {} ({})".format(
        kinect_to_mouse_delay, kinect_to_mouse_daytime
    )
)

# add the delay to the kinect markers timestamps
kinect_markers["time_stamps"] += kinect_to_mouse_delay

# put kinect and mouse markers in the same array
kinect_and_mouse_markers = []
for i in range(nbMouseMarkers):
    kinect_and_mouse_markers.append(
        [mouse_markers["time_stamps"][i], mouse_markers["time_series"][i][0]]
    )

for i in range(len(kinect_markers["time_stamps"])):
    kinect_and_mouse_markers.append(
        [kinect_markers["time_stamps"][i], kinect_markers["time_series"][i][0]]
    )

# sort the markers by timestamp
kinect_and_mouse_markers.sort(key=lambda x: x[0])

# compute the difference between two consecutive timestamps using np.diff
diff = np.diff([marker[0] for marker in kinect_and_mouse_markers])
# add a leading 0 to the diff array
diff = np.insert(diff, 0, 0)

# add the difference between two consecutive timestamps to the markers in the second column
for i in range(len(kinect_and_mouse_markers)):
    kinect_and_mouse_markers[i].append(diff[i])
    # permute the second and third columns (easier to read)
    kinect_and_mouse_markers[i][1], kinect_and_mouse_markers[i][2] = kinect_and_mouse_markers[i][2], kinect_and_mouse_markers[i][1]

# set all timestamps relative to the first timestamp
firstTimestamp = kinect_and_mouse_markers[0][0]
for i in range(len(kinect_and_mouse_markers)):
    kinect_and_mouse_markers[i][0] -= firstTimestamp

# print the markers
print("All markers")
for marker in kinect_and_mouse_markers:
    print("{:10.3f} {:10.3f}: {}".format(marker[0], marker[1], marker[2]))
    # print(marker)

# Compute the kinect-to-mouse delay

In [ ]:
def getKinectToMouseDelay(fullFname):
    """Get the delay between the kinect and the mouse data"""

    data, header = pyxdf.load_xdf(
        filename=fileName,
        synchronize_clocks=True,
        dejitter_timestamps=False,  # to get the raw timestamps to compare with the CSV
    )

    # find the mouse and kinect markers
    kinect_markers = [
        stream
        for stream in data
        if stream["info"]["name"][0] == "EuroMov-Markers-Kinect"
        and stream["info"]["type"][0] == "Markers"
    ]
    kinect_markers = kinect_markers[0]

    mouse_markers = [
        stream
        for stream in data
        if stream["info"]["name"][0] == "Mouse"
        and stream["info"]["type"][0] == "Markers"
    ]
    mouse_markers = mouse_markers[0]

    # check whether the mouse and kinect markers are from the same hostname
    kinect_hostname = kinect_markers["info"]["hostname"][0]
    mouse_hostname = mouse_markers["info"]["hostname"][0]
    if kinect_hostname == mouse_hostname:
        print(
            "The mouse and kinect markers are from the same computer clock: {}".format(
                kinect_hostname
            )
        )
    else:
        print(
            "CAUTION: the mouse and kinect markers are not from the same computer clock"
        )
        print("  Kinect hostname: {}".format(kinect_hostname))
        print("  Mouse hostname: {}".format(mouse_hostname))

    # get the delay
    kinect_delay = getKinectDelay(fileName)["delay"]
    mouse_delay = getMouseDelay(fileName)["delay"]
    kinect_to_mouse_delay = mouse_delay - kinect_delay

    # get the distribution of the delays
    kinect_delays_distribution = getKinectDelay(fileName)["delays_distribution"]
    mouse_delays_distribution = getMouseDelay(fileName)["delays_distribution"]
        
    # return a dict with the delays
    return {
        "kinect_delay": kinect_delay,
        "mouse_delay": mouse_delay,
        "kinect_to_mouse_delay": kinect_to_mouse_delay,
        "kinect_delays_distribution": kinect_delays_distribution,
        "mouse_delays_distribution": mouse_delays_distribution,
    }

fileName = goodFiles[8]
fileName  = goodFiles[17]

ktm = getKinectToMouseDelay(fileName)

print("The kinect to mouse delay is {} seconds".format(ktm["kinect_to_mouse_delay"]))
print("The kinect delay is {} seconds".format(ktm["kinect_delay"]))
print("The mouse delay is {} seconds".format(ktm["mouse_delay"]))
print("The kinect delay distribution is {}".format(ktm["kinect_delays_distribution"]))
print("The mouse delay distribution is {}".format(ktm["mouse_delays_distribution"]))
print("The kinect delay distribution median is {}".format(np.median(ktm["kinect_delays_distribution"])))
print("The mouse delay distribution median is {}".format(np.median(ktm["mouse_delays_distribution"])))

# make a boxplot of the distribution of the delays
fig = plt.figure()
plt.boxplot([ktm["kinect_delays_distribution"], ktm["mouse_delays_distribution"]], vert=False, labels=["Kinect", "Mouse"])
plt.xlabel("Delay (s)")
plt.title("Distribution of the delays")
plt.show()

# 22851.839111089706
# 21495.123371124268

# Check that the kinect-to-mouse delay can correct the timestamps of the kinect streams

## read the marker file 

In [ ]:
fnameMarkerCSV = "/Users/denismottet/Documents/GitHub/Rearm-CheckFiles/dat/ReArm.lnk/ReArm_C1P02/Circle/ReArm_C1P02_20210322_1_c_l_m_np.csv"

with open(fnameMarkerCSV, "r") as fname:
    txt = fname.readlines()

# # find the index of the lines starting with a digit and ending with a comma
# lines_ending_with_comma = []
# for i in range(len(lines)):
#     if lines[i][0].isdigit() and lines[i][-2] == ",":
#         lines_ending_with_comma.append([i, lines[i]])


# split each line into tokens separated by a comma
lines = [line.split(",") for line in txt]

# find the lines where token 3 is "\n" = start of a multiline marker
for i in range(len(lines)):
    line = lines[i]
    if len(line) == 3 and line[2] == "\n":
        # replace the end of the line by a " (start of a multiline marker)
        txt[i] = txt[i][:-1] + '"'
        # find the end of the multiline marker
        for j in range(i + 1, len(lines)):
            if len(lines[j]) == 3 :
                # add a " before the end of the line
                txt[j-1] = txt[j-1][:-1] + '"\n'
                break
            if j == len(lines) - 1 and len(lines[j]) != 3:
                txt[j] = txt[j][:-1] + '"\n'
                break

# save lines to a temporary file using tempfile
import tempfile

with tempfile.NamedTemporaryFile(mode="w", delete=False) as fname:
    fname.writelines(txt)
    tempFileName = fname.name

# print the temporary file content
# print()
# print("Temporary file content:")
# with open(tempFileName, "r") as fname:
#     print(fname.read())

# read the temporary file as a numpy array
lines = np.loadtxt(tempFileName, skiprows=3, delimiter=",", quotechar = '"', dtype=str)

print()
# print the lines
for line in lines:
    print(line)


## Create a function to read the marker file

In [ ]:
def readMarkerFile(fnameMarkerCSV): 

    with open(fnameMarkerCSV, "r") as fname:
        txt = fname.readlines()

    lines = [line.split(",") for line in txt]

    # add the timestamp column if we have only 2 columns
    for i in range(len(lines)):
        if len(lines[i]) == 2 and lines[i][0][0].isdigit():
            # create a timestamp that is in milliseconds (as in mouse markers)
            timestamp = datetime.strptime(lines[i][0], "%Y-%m-%d %H:%M:%S.%f").timestamp() * 1000 
            lines[i].insert(1, str(timestamp))
            txt[i] = ",".join(lines[i]) 

    # find the lines where token 3 is "\n" = start of a multiline marker
    for i in range(len(lines)):
        line = lines[i]
        if len(line) == 3 and line[2] == "\n":
            # replace the end of the line by a " (start of a multiline marker)
            txt[i] = txt[i][:-1] + '"\n'
            # find the end of the multiline marker
            for j in range(i + 1, len(lines)):
                if len(lines[j]) == 3 :
                    # add a " before the end of the line
                    txt[j-1] = txt[j-1][:-1] + '"\n'
                    break
                if j == len(lines) - 1 and len(lines[j]) != 3:
                    txt[j] = txt[j][:-1] + '"\n'
                    break

    import tempfile
    with tempfile.NamedTemporaryFile(mode="w", delete=False) as fname:
        fname.writelines(txt)
        tempFileName = fname.name

    lines = np.loadtxt(tempFileName, skiprows=3, delimiter=",", quotechar = '"', dtype=str)

    # remove the first column (we shall need only the timestamp and the marker to compare with the xdf file)
    lines = np.delete(lines, 0, 1)
    # change the type of the timestamp column to float
    out = []
    for line in lines:
        # CAUTION : the timestamp must be in seconds
        out.append([float(line[0])/1000, line[1]])

        

    return out

fnameMarkerCSV = "/Users/denismottet/Documents/GitHub/Rearm-CheckFiles/dat/ReArm.lnk/ReArm_C1P02/Circle/ReArm_C1P02_20210322_1_c_l_m_np.csv"
fnameMarkerCSV = "/Users/denismottet/Documents/GitHub/Rearm-CheckFiles/dat/ReArm.lnk/ReArm_C1P02/Circle/ReArm_C1P02_20210322_1_c_k_m.csv"
#fnameMarkerCSV = "/Users/denismottet/Documents/GitHub/Rearm-CheckFiles/dat/ReArm.lnk/ReArm_C1P02/Reaching/ReArm_C1P02_20210322_1_r_k_m.csv"

lines = readMarkerFile(fnameMarkerCSV)

# print the lines
for line in lines:
    print(line)

## set the expected marker sequence from the csv files 


In [ ]:
fileName = goodFiles[17]
#fileName = goodFiles[8]


def setExpectedMarkerSequence(fileName):
    fpath, fname, extension = file_parts(fileName)

    # find all the csv files in the directory that are markers files
    import glob

    csvFiles = glob.glob("*_m*.csv", root_dir=fpath)

    expectedMarkerSequence = []
    for foundFile in csvFiles:
        fullFname_csv = os.path.join(fpath, foundFile)
        data_csv = readMarkerFile(fullFname_csv)
        for line in data_csv:
            expectedMarkerSequence.append(line)

    expectedMarkerSequence.sort(key=lambda x: x[0])

    return expectedMarkerSequence, csvFiles


expectedMarkerSequence, csvFiles = setExpectedMarkerSequence(fileName)

# # print the timestamps
print("expectedMarkerSequence:")
for i in range(len(expectedMarkerSequence)):
    print(
        "  {}: {}".format(expectedMarkerSequence[i][0], expectedMarkerSequence[i][-1])
    )

# Get the actual marker sequence from the XDF file

In [ ]:
fileName = goodFiles[8]
fileName  = goodFiles[17]

def getActualMarkerSequence(fileName):

    # load the whole xdf file
    data, header = pyxdf.load_xdf(
        filename=fileName,
        synchronize_clocks=True,
        dejitter_timestamps=False,  # to get the raw timestamps to compare with the CSV
    )

    # get the kinedt-to-mouse delay
    ktm = getKinectToMouseDelay(fileName)
    kinect_to_mouse_delay = ktm["kinect_to_mouse_delay"]

    # make correction to the timestamps of the kinect streams
    for stream in data:
        if "Kinect" in stream["info"]["name"][0]: 
            stream["time_stamps"] += kinect_to_mouse_delay
            if stream["info"]["type"][0] == "Markers":
                if "Initial stop" in stream["time_series"][0][0]:
                    # remove the "Initial stop" marker
                    stream["time_stamps"] = stream["time_stamps"][1:]
                    stream["time_series"] = stream["time_series"][1:]


    # find the mouse and kinect markers
    actualMarkerSequence = []
    for stream in data:
        #if ("Mouse" in stream["info"]["name"][0] or "Kinect" in stream["info"]["name"][0]  )and "Markers" in stream["info"]["type"][0] :
        if "Markers" in stream["info"]["type"][0] and (stream["info"]["name"][0] == "Mouse" or stream["info"]["name"][0] == "EuroMov-Markers-Kinect"):
            print("{}: {}".format(stream["info"]["name"][0], stream["info"]["type"][0]))
            for i in range(len(stream["time_stamps"])):
                actualMarkerSequence.append(
                    [
                        stream["time_stamps"][i],
                        stream["time_series"][i][0],
                        stream["info"]["name"][0]
                    ]
                )

    # sort the markers by timestamp
    actualMarkerSequence.sort(key=lambda x: x[0])

    return actualMarkerSequence

actualMarkerSequence = getActualMarkerSequence(fileName)

# print the markers
print("Actual marker sequence:")
for marker in actualMarkerSequence:
    print(marker)

# Compare the expected marker sequence with the actual marker sequence

In [ ]:
fileName = goodFiles[8]
fileName  = goodFiles[17]

actualMarkerSequence = getActualMarkerSequence(fileName)
expectedMarkerSequence, csvFiles = setExpectedMarkerSequence(fileName)

# make the timestamps relative to the first timestamp
firstTimestamp = actualMarkerSequence[0][0]
for i in range(len(actualMarkerSequence)):
    actualMarkerSequence[i][0] -= firstTimestamp

firstTimestamp = expectedMarkerSequence[0][0]
for i in range(len(expectedMarkerSequence)):
    expectedMarkerSequence[i][0] -= firstTimestamp
    
print(":")
print("actualMarkerSequence:")
print(len(actualMarkerSequence) )
for i in range(len(actualMarkerSequence)):
    print("{:10.3f}, {}".format(actualMarkerSequence[i][0], actualMarkerSequence[i][1]) )

print(":")
print("expectedMarkerSequence:")
print(len(expectedMarkerSequence))
for i in range(len(expectedMarkerSequence)):
    print("{:10.3f}, {}".format(expectedMarkerSequence[i][0], expectedMarkerSequence[i][1]) )    
# print the first 10 markers, with the  actual and expected sequences on the same line
print(":")
print("First markers:")
for i in range(27):
    print(
        actualMarkerSequence[i][0]  , actualMarkerSequence[i][1], expectedMarkerSequence[i][0] , expectedMarkerSequence[i][1]   
    )



In [ ]:
# check line by line whether the expected sequences is the same as the actual sequence
for i in range(len(expectedMarkerSequence)):
    if actualMarkerSequence[i][1] != expectedMarkerSequence[i][1]:
        print("Found a correct sequence of {} markers out of {} expected".format(i, len(expectedMarkerSequence)))
        break

# check whether the actual and expected correct sequences have the similar diff between two consecutive timestamps
actualDiff = np.diff([marker[0] for marker in actualMarkerSequence[:i]])
expectedDiff = np.diff([marker[0] for marker in expectedMarkerSequence[:i]])

errorinDiff = actualDiff - expectedDiff

# make a boxplot of the distribution of the delays
fig = plt.figure()
plt.boxplot( (actualDiff - expectedDiff)*1000, vert=False, labels=["Diff "])
plt.xlabel("Error (milliseconds)")
plt.title("Distribution of actualDiff - expectedDiff")
plt.show()

# print the max error 
print("The max error is {:3.2f} milliseconds".format(np.max(np.abs(errorinDiff))*1000))

# check actualDiff and expectedDiff never differ by more than 5 ms
time_error_tolerance = 0.005
if np.allclose(actualDiff, expectedDiff, atol=time_error_tolerance):
    print("The actual and expected sequences have the same timing (tolereance = {} ms )".format(time_error_tolerance*1000))
    


# # check whether the actual and expected sequences have the same diff between two consecutive timestamps
# actualDiff = np.diff([marker[0] for marker in actualMarkerSequence])
# expectedDiff = np.diff([marker[0] for marker in expectedMarkerSequence])
# if np.array_equal(actualDiff, expectedDiff):
#     print("The actual and expected sequences have the same diff between two consecutive timestamps")
# else:
#     print("Error: the actual and expected sequences do not have the same diff between two consecutive timestamps")
#     print("  actual: {}".format(actualDiff))
#     print("  expected: {}".format(expectedDiff))

## Run the kinect-to-mouse delay correction

In [ ]:
fileName = goodFiles[17]

# load the whole xdf file
data, header = pyxdf.load_xdf(
    filename=fileName,
    synchronize_clocks=True,
    dejitter_timestamps=False,  # to get the raw timestamps to compare with the CSV
)

ktm = getKinectToMouseDelay(fileName)

# get the kinect streams to be corrected
kinect_streams = [
    stream
    for stream in data
    if "Kinect" in stream["info"]["name"][0] 
]

print("Found {} kinect streams".format(len(kinect_streams)))

# make the correction to the timestamps of the kinect streams
for stream in kinect_streams:
    stream["time_stamps"] += ktm

# save the corrected data to a new file using pickle
# import pickle
# with open("data.pickle", "wb") as f:
#     pickle.dump(data, f)

# get the mouse streams (for the plot)
mouse_streams = [
    stream
    for stream in data
    if "Mouse" in stream["info"]["name"][0] 
]
print("Found {} mouse streams".format(len(mouse_streams)))


# plot the kinect timestamps and the mouse timestamps
fig = plt.figure()
for i in range(len(kinect_streams)):
    stream = kinect_streams[i]
    # plot a line from the fist and last timestamps 
    plt.plot(
        [stream["time_stamps"][0], stream["time_stamps"][-1]],
        [i, i],
        label=stream["info"]["name"][0],
    )
    print(stream["info"]["name"][0], stream["time_stamps"][0], stream["time_stamps"][-1])
for i in range(len(mouse_streams)):
    stream = mouse_streams[i]
    # plot a line from the fist and last timestamps 
    plt.plot(
        [stream["time_stamps"][0], stream["time_stamps"][-1]],
        [i+10, i+10],
        label=stream["info"]["name"][0] + stream["info"]["type"][0],
    )
    print(stream["info"]["name"][0], stream["time_stamps"][0], stream["time_stamps"][-1])
plt.legend()

plt.xlabel("Time (s)")
plt.ylabel("Stream")
plt.show()


# Get the kinect-to-mouse delay for all xdf files 

In [ ]:
for file in goodFiles:
    if file.endswith("xdf"):
        ktm = getKinectToMouseDelay(file)
        print("{}: {}".format(file, ktm))

## Print the info about each stream 

In [ ]:
import os
import pyxdf
import numpy as np
import matplotlib.pyplot as plt


def print_all_streamInfo(data):
    """
    Prints the info of each stream in the XDF file.
    """

    shiftBy = "  "

    for stream in data:
        print(stream["info"]["name"][0])
        for k, v in stream["info"].items():
            printList(shiftBy, k, v)


def printList(shift, name, myList):
    """
    Prints a list (of array of list) with str items.
    The function recurs until it finds a leaf that is a str.
    Each recurrence is shifted to account for the structure.
    """

    def printLeaf(shift, name, item):
        print(shift, name, item)

    if isinstance(myList, list):
        for i in range(0, len(myList)):
            if isinstance(myList[i], str):
                printLeaf(shift, name, myList[i])
            else:
                name_i = name + "[" + str(i) + "]"
                print(shift, name_i)
                if myList[i] is not None:
                    for vi, ki in myList[i].items():
                        printList(shift + shift, vi, ki)
    else:
        printLeaf(shift, name, myList)


####################  MAIN  ####################

xdf_fullFname = goodFiles[8]  # xdf file
data, header = pyxdf.load_xdf(xdf_fullFname)
print_all_streamInfo(data)